In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np

# Import our utilities functions
import utils

## Prepare the Data

In [2]:
# Get the original DataFrame
df = utils.get_data()
df.head()

,user_id,movie_id,rating
0,1,1,2.0
1,1,2,4.0
2,1,3,3.5
3,1,4,3.0
4,1,5,4.0


In [3]:
# Get the normalized ratings
normalized_ratings = utils.get_normalized_data()
normalized_ratings.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.4,0.8,0.7,0.6,0.8,0.7,0.7,0.6,0.5,0.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0.0,0.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.6,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
# Get the weights and bias tensors
W = utils.weights()
hb = utils.hidden_bias()
vb = utils.visible_bias()

In [5]:
# Test generating user recommendation
test_user = 1024

# Get user ratings
user_ratings = normalized_ratings.loc[test_user]

# Generate recommendations for user
rec = utils.generate_recommendation(user_ratings, W, vb, hb)

# Construct user recommendation DataFrame
user_recommendation = pd.DataFrame({"movie_id": normalized_ratings.columns, "user_id": test_user})
user_recommendation = user_recommendation.assign(RecommendationScore = rec[0].numpy())

# View sorted user recommendation
user_recommendation.sort_values(["RecommendationScore"], ascending=False).head(20)

,movie_id,user_id,RecommendationScore
214,215,1024,0.924097
10,11,1024,0.857634
211,212,1024,0.802491
8,9,1024,0.779303
12,13,1024,0.739550
4,5,1024,0.738329
251,252,1024,0.734466
9,10,1024,0.725765
253,254,1024,0.702583
210,211,1024,0.699687


In [6]:
# Merge user recommendation scores with original dataset ratings
merged_df = user_recommendation.merge(df, on=['movie_id', 'user_id'], how='outer')
merged_df.sort_values(["RecommendationScore"], ascending=False).head(20)

,movie_id,user_id,RecommendationScore,rating
17370,215,1024,0.924097,4.0
8058,11,1024,0.857634,3.5
15671,212,1024,0.802491,0.5
6465,9,1024,0.779303,3.0
9656,13,1024,0.739550,3.5
3647,5,1024,0.738329,NaN
27267,252,1024,0.734466,NaN
7183,10,1024,0.725765,2.5
28145,254,1024,0.702583,4.0
15205,211,1024,0.699687,NaN


In [7]:
merged_df['PredictedRating'] = merged_df['RecommendationScore'] * 5

In [9]:
merged_df.sort_values(["RecommendationScore"], ascending=False).head(20)

,movie_id,user_id,RecommendationScore,rating,PredictedRating
17370,215,1024,0.924097,4.0,4.620483
8058,11,1024,0.857634,3.5,4.288169
15671,212,1024,0.802491,0.5,4.012457
6465,9,1024,0.779303,3.0,3.896516
9656,13,1024,0.739550,3.5,3.697748
3647,5,1024,0.738329,NaN,3.691643
27267,252,1024,0.734466,NaN,3.672330
7183,10,1024,0.725765,2.5,3.628827
28145,254,1024,0.702583,4.0,3.512913
15205,211,1024,0.699687,NaN,3.498437
